In [30]:
import numpy as np
from pele.potentials import LJ
from pele.utils import xyz
from pele.optimize import lbfgs_py
#from .tip4p_system import TIP4PSystem

# read in coordinates from xyz file
ref = xyz.read_xyz(open("TIP4P-16.xyz"))
xyz.write_xyz(open("test.xyz", "w"), coords=ref.coords)
# lookup table for atom masses
mass_lookup = {'O': 16., 'H': 1.}

#ref.coords[:] *= 3


In [32]:
import numpy as np
from copy import deepcopy
from pele.angleaxis import RBTopology, RBSystem
 # For rigid fragment
import numpy as np
from pele.potentials import LJ
from pele.utils import xyz
from pele.angleaxis import rigidbody
from pele.optimize import lbfgs_py
 
# read in coordinates from xyz file
ref = xyz.read_xyz(open("TIP4P-16.xyz"))
xyz.write_xyz(open("test.xyz", "w"), coords=ref.coords)
# lookup table for atom masses
mass_lookup = {'O': 16., 'H': 1.}


def water(xyzfile): # TIP4P-16.xyz"
    ref = xyz.read_xyz(open(xyzfile))
    xyz.write_xyz(open("test.xyz", "w"), coords=ref.coords)
# lookup table for atom masses
    mass_lookup = {'O': 16., 'H': 1.}
    rb_sites = []
    for atomtype, x, i in zip(ref.atomtypes, ref.coords, xrange(len(ref.atomtypes))):
        # every 3rd atom, define a new reigid molecule
        if i % 3 == 0:
            rb = rigidbody.RigidFragment()
            rb_sites.append(rb)
        rb.add_atom(atomtype, x, mass_lookup[atomtype])
# finalize the rigid body setup
    for rb in rb_sites:
        rb.finalize_setup()
    return rb_sites, ref 

class TIP4PSystem(RBSystem):
    def __init__(self, xyzfile):
        self.xyzfile = xyzfile
        RBSystem.__init__(self)
    def setup_aatopology(self):
        water_sites, ref = water(self.xyzfile)
        rbsystem = RBTopology()
        rbsystem.add_sites(water_sites)
        rbsystem.finalize_setup()
        #print len(rbsystem.sites), len(rbsystem.indices)
        print "I have %d water molecules in the system" % len(rbsystem.sites)
        rbcoords = rbsystem.coords_adapter(np.zeros(len(rbsystem.sites)*6))
        for site, com in zip(rbsystem.sites, rbcoords.posRigid):
            com[:] = ref.coords[site.atom_indices[0]] - site.atom_positions[0]
        pot = LJ(eps=0.1550, sig=3.1536)
        # get the flattened coordinate array
        print "The initial energy is", pot.getEnergy(ref.coords.flatten())
        rbpot = rigidbody.RBPotentialWrapper(rbsystem, pot) 
        print "rbpot.getEnergy(rbcoords.coords)", rbpot.getEnergy(rbcoords.coords)
        e, g = rbpot.getEnergyGradient(rbcoords.coords)
        g_n = rbpot.NumericalDerivative(rbcoords.coords, eps=1e-4)
        cg = rbsystem.coords_adapter(g-g_n)

        # coords = rbpot.getCoords()
        # nrigid = rbcoords.size / 6
        # print "nrigid", nrigid
        self.potential = rbpot
        self.nrigid = len(rbsystem.sites)
        self.render_scale = 0.3
        self.atom_types = rbsystem.get_atomtypes()
        self.draw_bonds = []
        for i in xrange(self.nrigid):
            self.draw_bonds.append((3*i, 3*i+1))
            self.draw_bonds.append((3*i, 3*i+2))
        rbsystem.finalize_setup()
        print "final!"
        return rbsystem

    def get_potential(self):
        return self.potential


In [33]:
#tip4p 16 
system = TIP4PSystem('TIP4P-16.xyz')
from pele import storage 

will compute the lowest eigenvector by diagonalizing the Hessian
I have 16 water molecules in the system
The initial energy is 32502468.4518
rbpot.getEnergy(rbcoords.coords) 32502468.4518
final!


In [34]:
print system.get_masses()
print system.get_permlist()

AttributeError: 'TIP4PSystem' object has no attribute 'get_masses'

In [35]:
#%xmode

In [36]:

def compare_min():
    raise NotImplementedError 
    return 
db = system.create_database(db='tip4p16.sqlite')

In [37]:
bh = system.get_basinhopping(database=db, outstream=None)
bh.run(100)

In [38]:
print "number of minima", db.number_of_minima()
print "number of transition states", db.number_of_transition_states()

number of minima 146
number of transition states 0


In [39]:
import logging
logger = logging.getLogger("pele.connect")
logger.setLevel("WARNING")

In [ ]:
from pele.landscape import ConnectManager
manager = ConnectManager(db)
for i in xrange(20):
    min1, min2 = manager.get_connect_job()
    connect = system.get_double_ended_connect(min1, min2, db, verbosity=-1)
    connect.connect()

sending a random connect job 138 119
no reinterpolation needed
no reinterpolation needed
will compute the lowest eigenvector by diagonalizing the Hessian
lbfgs: having trouble finding a good step size. 1.0000000000000002e-12 6.18814738813385, resetting the minimizer
lbfgs: having trouble finding a good step size. 1e-12 0.12005794382824535, resetting the minimizer
lbfgs: having trouble finding a good step size. 1.0000000000000002e-12 0.17020071524425265, resetting the minimizer
lbfgs: having trouble finding a good step size. 1.0000000000000002e-12 0.17020071524425265, resetting the minimizer
lbfgs: having trouble finding a good step size. 1.0000000000000002e-12 0.17020071524425265, resetting the minimizer
lbfgs: having trouble finding a good step size. 1.0000000000000002e-12 0.17020071524425265, resetting the minimizer
lbfgs: having trouble finding a good step size. 1.0000000000000002e-12 0.17020071524425265, resetting the minimizer
lbfgs: having trouble finding a good step size. 1.0000

In [ ]:
print "number of minima", db.number_of_minima()
print "number of transition states", db.number_of_transition_states()

In [ ]:
from pele.utils.disconnectivity_graph import DisconnectivityGraph, database2graph

graph = database2graph(db)
dgraph = DisconnectivityGraph(graph)
dgraph.calculate()
dgraph.plot()
import matplotlib.pyplot as plt
%matplotlib inline
# plt.figure(figsize=(30,1))
plt.show()